In [ ]:
#### import global modules
import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path
from yaml import safe_load
import google.oauth2.credentials
from google.cloud import bigquery
import gc

# Set global vars
pth_project = Path(os.getcwd().split('notebooks')[0])
pth_data = pth_project / 'data'
pth_queries = pth_project / 'core' / 'queries'
pth_creds = pth_project / 'conf' / 'local' / 'project_config.yaml'
sys.path.insert(0, str(pth_project))
d_project_config = safe_load(pth_creds.open())
# d_params = safe_load((pth_project / 'core' / 'parameters' / 'common.yaml').open())['data_extract']

# import local modules
from core.utils.gcp import connect_bq_services
# from core.etl.extract import extract_bq_data, extract_pr_codes, format_conv_df, filter_convs

# Connect to google services
bq_client = connect_bq_services(d_project_config['gcp-project-name'])
pd.options.display.max_rows = 100

In [ ]:
def extract_bq_data(bq_client, sql=None, pth_query=None):
    if sql is not None:
        df = bq_client.query(sql).to_dataframe()
    elif pth_query is not None:
        sql = pth_query.read_text()
        df = bq_client.query(sql).to_dataframe()
    else:
        raise ValueError('`sql` or `pth_query` should be set')  
    return df

In [ ]:
Campaign_Query='''


select *
from `divgpras-pr-579355.SHS.SHS_ALR_Campaign_data_SAS`
where REGEXP_CONTAINS (CAMP_CREATIVE, r'ALR1_TP5')
and date(Campaign_date)>='2022-05-01' and date(Campaign_date)<='2022-08-31'

'''

In [ ]:
Camp_DF=extract_bq_data(bq_client, sql=Campaign_Query)

In [ ]:
Camp_DF.info()

In [ ]:
def Camp_creative_making(row):
    
    if row['CAMP_CREATIVE']=='ALR1_TP5X_V1' or row['CAMP_CREATIVE']=='ALR1_TP5_V1':
        return 'V1'
    elif row['CAMP_CREATIVE']=='ALR1_TP5X_V2' or row['CAMP_CREATIVE']=='ALR1_TP5_V2':
        return 'V2'
    elif row['CAMP_CREATIVE']=='ALR1_TP5X_V3' or row['CAMP_CREATIVE']=='ALR1_TP5_V3':
        return 'V3'
    elif row['CAMP_CREATIVE']=='ALR1_TP5X_V4' or row['CAMP_CREATIVE']=='ALR1_TP5_V4':
        return 'V4'
    else:
        return "None"

In [ ]:
Camp_DF['CAMP_CREATIVE'].value_counts().sort_index()

In [ ]:
Camp_DF['Campaign_date'].value_counts().sort_index()

In [ ]:
Camp_DF['CAMP_CREATIVE_New']=Camp_DF.apply(Camp_creative_making,axis=1)

In [ ]:
Camp_DF['CAMP_CREATIVE_New'].value_counts().sort_index()

In [ ]:
# Camp_DF_May_Sep22=Camp_DF[Camp_DF.Campaign_date<='2022-09-30' and Camp_DF.Campaign_date>='2022-05-01']

In [ ]:
# Camp_DF['Campaign_date'].value_counts().sort_index()

In [ ]:
# Camp_DF_Mar2022['CAMP_CREATIVE_New'].value_counts().sort_index()

In [ ]:
pd.DataFrame(Camp_DF.groupby(['CAMP_CREATIVE_New','OPENED']).agg(
    
    Customer_count= ('BACCT_NUM','nunique'),
    Email_Opened_count=('OPENED','sum'),
    CT_Opened_count=('CLICKTHROUGH','sum'),
    softbounce_count=('SOFTBOUNCE','sum'),
    Hardbounce_count=('HARDBOUNCE','sum'),
    Unsub_count=('UNSUBSCRIBE','sum'),
    campaign_date_min=('Campaign_date','min'),
    campaign_date_max=('Campaign_date','max')

    
    # # Customer_Share= ('customer_id',lambda x:x.count()*100/Merge_DF_4.shape[0])
    # Churn_total=('Telus_Churn_Flag',lambda x: x.sum()),
    # Churn_rate=('Telus_Churn_Flag',lambda x: x.mean()*100)
).reset_index())

In [ ]:
# config= bigquery.job.LoadJobConfig()
# config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

# Table_BQ = 'SHS.SHS_ALR_Campaign_data_Jan_June_2022'

# bq_table_instance= bq_client.load_table_from_dataframe(Camp_DF, Table_BQ,job_config=config)

In [ ]:
Sep_Snapshot='''



DECLARE target_date DATE DEFAULT "2022-09-01";


With Camp_V1_BAN as (

select distinct BACCT_NUM as BAN,CAMP_CREATIVE_New from `divgpras-pr-579355.SHS.SHS_ALR_Campaign_data_May_Aug_2022`

)


, prod_snpsht_SMHM as (


select distinct bacct_bus_bacct_num as BAN,1 as count_of_BAN_FFH
FROM `cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht` 
where pi_prod_instnc_typ_cd='SMHM' 
and pi_prod_instnc_stat_cd in ('A')
and  consldt_cust_typ_cd = 'R'
and DATE(prod_instnc_ts)=target_date


)

select a.CAMP_CREATIVE_New, sum(b.count_of_BAN_FFH) as Distinct_BAN_count
 from Camp_V1_BAN a
inner join prod_snpsht_SMHM b
on a.BAN=b.BAN
group by CAMP_CREATIVE_New
order by CAMP_CREATIVE_New


'''

In [ ]:
Sep_Snapshot_count=extract_bq_data(bq_client, sql=Sep_Snapshot)

In [ ]:
Sep_Snapshot_count

In [ ]:
Feb2023_Snapshot='''



DECLARE target_date DATE DEFAULT "2023-02-01";


With Camp_V1_BAN as (

select distinct BACCT_NUM as BAN,CAMP_CREATIVE_New from `divgpras-pr-579355.SHS.SHS_ALR_Campaign_data_May_Aug_2022`

)


, prod_snpsht_SMHM as (


select distinct bacct_bus_bacct_num as BAN,1 as count_of_BAN_FFH
FROM `cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht` 
where pi_prod_instnc_typ_cd='SMHM' 
and pi_prod_instnc_stat_cd in ('A')
and  consldt_cust_typ_cd = 'R'
and DATE(prod_instnc_ts)=target_date


)

select a.CAMP_CREATIVE_New, sum(b.count_of_BAN_FFH) as Distinct_BAN_count
 from Camp_V1_BAN a
inner join prod_snpsht_SMHM b
on a.BAN=b.BAN
group by CAMP_CREATIVE_New
order by CAMP_CREATIVE_New


'''

In [ ]:
Feb2023_Snapshot_count=extract_bq_data(bq_client, sql=Feb2023_Snapshot)

In [ ]:
Feb2023_Snapshot_count

In [ ]:
Sep_Snapshot='''

DECLARE target_date_start DATE DEFAULT "2022-08-01";
DECLARE target_date_end DATE DEFAULT "2022-09-01";



With Camp_V1_BAN as (

select distinct BACCT_NUM as BAN,CAMP_CREATIVE_New from `divgpras-pr-579355.SHS.SHS_ALR_Campaign_data_May_Aug_2022`

)


, prod_snpsht_SMHM as (


select distinct bacct_bus_bacct_num as BAN,1 as count_of_BAN_FFH
FROM `cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht` 
where pi_prod_instnc_typ_cd='SMHM' 
and pi_prod_instnc_stat_cd in ('A')
and  consldt_cust_typ_cd = 'R'
and DATE(prod_instnc_ts)=target_date_end


)

, ARPU_Calculation as(
SELECT
  distinct billg_acct_num AS ban,
  EXTRACT(YEAR
  FROM
    bill.bill_dt)||'-'||LPAD(CAST(EXTRACT(MONTH
      FROM
        bill.bill_dt) AS STRING), 2, '0') AS bill_year_month,
  sum(bill.rid_unit_chrg_amt) as ARPU_SMHM,
  sum(tot_inv_amt) as ARPU_FFH
  FROM
  `cio-datahub-enterprise-pr-183a.ent_cust_bill.bq_wln_inv_sum_view` bill
WHERE
  DATE(bill.bill_dt)>=  target_date_start
  and DATE(bill.bill_dt)< target_date_end
  group by ban,bill_year_month
)

select *
 from Camp_V1_BAN a
inner join prod_snpsht_SMHM b
on a.BAN=b.BAN
left join ARPU_Calculation ARPU_df
on a.BAN=ARPU_df.ban


'''

In [ ]:
Sep_Snapshot_DF=extract_bq_data(bq_client, sql=Sep_Snapshot)

In [ ]:
Sep_Snapshot_DF.info()

In [ ]:
pd.DataFrame(Sep_Snapshot_DF.groupby(['CAMP_CREATIVE_New']).agg(
    
    Customer_count= ('count_of_BAN_FFH','sum'),
    AVG_ARPU_FFH=('ARPU_FFH','mean'),
    AVG_ARPU_SMHM=('ARPU_SMHM','mean'),
    
    # # Customer_Share= ('customer_id',lambda x:x.count()*100/Merge_DF_4.shape[0])
    # Churn_total=('Telus_Churn_Flag',lambda x: x.sum()),
    # Churn_rate=('Telus_Churn_Flag',lambda x: x.mean()*100)
).reset_index())

In [ ]:
Feb2023_Snapshot='''

DECLARE target_date_start DATE DEFAULT "2023-01-01";
DECLARE target_date_end DATE DEFAULT "2023-02-01";



With Camp_V1_BAN as (

select distinct BACCT_NUM as BAN,CAMP_CREATIVE_New from `divgpras-pr-579355.SHS.SHS_ALR_Campaign_data_May_Aug_2022`

)


, prod_snpsht_SMHM as (


select distinct bacct_bus_bacct_num as BAN,1 as count_of_BAN_FFH
FROM `cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht` 
where pi_prod_instnc_typ_cd='SMHM' 
and pi_prod_instnc_stat_cd in ('A')
and  consldt_cust_typ_cd = 'R'
and DATE(prod_instnc_ts)=target_date_end


)

, ARPU_Calculation as(
SELECT
  distinct billg_acct_num AS ban,
  EXTRACT(YEAR
  FROM
    bill.bill_dt)||'-'||LPAD(CAST(EXTRACT(MONTH
      FROM
        bill.bill_dt) AS STRING), 2, '0') AS bill_year_month,
  sum(bill.rid_unit_chrg_amt) as ARPU_SMHM,
  sum(tot_inv_amt) as ARPU_FFH
  FROM
  `cio-datahub-enterprise-pr-183a.ent_cust_bill.bq_wln_inv_sum_view` bill
WHERE
  DATE(bill.bill_dt)>=  target_date_start
  and DATE(bill.bill_dt)< target_date_end
  group by ban,bill_year_month
)

select *
 from Camp_V1_BAN a
inner join prod_snpsht_SMHM b
on a.BAN=b.BAN
left join ARPU_Calculation ARPU_df
on a.BAN=ARPU_df.ban


'''

In [ ]:
Feb_Snapshot_DF=extract_bq_data(bq_client, sql=Feb2023_Snapshot)

In [ ]:
pd.DataFrame(Feb_Snapshot_DF.groupby(['CAMP_CREATIVE_New']).agg(
    
    Customer_count= ('count_of_BAN_FFH','sum'),
    AVG_ARPU_FFH=('ARPU_FFH','mean'),
    AVG_ARPU_SMHM=('ARPU_SMHM','mean'),
    
    # # Customer_Share= ('customer_id',lambda x:x.count()*100/Merge_DF_4.shape[0])
    # Churn_total=('Telus_Churn_Flag',lambda x: x.sum()),
    # Churn_rate=('Telus_Churn_Flag',lambda x: x.mean()*100)
).reset_index())

In [ ]:
May2022_Snapshot='''

DECLARE target_date_start DATE DEFAULT "2022-04-01";
DECLARE target_date_end DATE DEFAULT "2022-05-01";



With Camp_V1_BAN as (

select distinct BACCT_NUM as BAN,CAMP_CREATIVE_New from `divgpras-pr-579355.SHS.SHS_ALR_Campaign_data_May_Aug_2022`

)


, prod_snpsht_SMHM as (


select distinct bacct_bus_bacct_num as BAN,1 as count_of_BAN_FFH
FROM `cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht` 
where pi_prod_instnc_typ_cd='SMHM' 
and pi_prod_instnc_stat_cd in ('A')
and  consldt_cust_typ_cd = 'R'
and DATE(prod_instnc_ts)=target_date_end


)

, ARPU_Calculation as(
SELECT
  distinct billg_acct_num AS ban,
  EXTRACT(YEAR
  FROM
    bill.bill_dt)||'-'||LPAD(CAST(EXTRACT(MONTH
      FROM
        bill.bill_dt) AS STRING), 2, '0') AS bill_year_month,
  sum(bill.rid_unit_chrg_amt) as ARPU_SMHM,
  sum(tot_inv_amt) as ARPU_FFH
  FROM
  `cio-datahub-enterprise-pr-183a.ent_cust_bill.bq_wln_inv_sum_view` bill
WHERE
  DATE(bill.bill_dt)>=  target_date_start
  and DATE(bill.bill_dt)< target_date_end
  group by ban,bill_year_month
)

select *
 from Camp_V1_BAN a
inner join prod_snpsht_SMHM b
on a.BAN=b.BAN
left join ARPU_Calculation ARPU_df
on a.BAN=ARPU_df.ban


'''

In [ ]:
May2022_Snapshot_DF=extract_bq_data(bq_client, sql=May2022_Snapshot)

In [ ]:
pd.DataFrame(May2022_Snapshot_DF.groupby(['CAMP_CREATIVE_New']).agg(
    
    Customer_count= ('count_of_BAN_FFH','sum'),
    AVG_ARPU_FFH=('ARPU_FFH','mean'),
    AVG_ARPU_SMHM=('ARPU_SMHM','mean'),
    
    # # Customer_Share= ('customer_id',lambda x:x.count()*100/Merge_DF_4.shape[0])
    # Churn_total=('Telus_Churn_Flag',lambda x: x.sum()),
    # Churn_rate=('Telus_Churn_Flag',lambda x: x.mean()*100)
).reset_index())

In [ ]:
Contract_status_Query='''




with Intial_snapshot as (
        
        SELECT 
     distinct b.BACCT_NUM as ban
    ,b.CAMP_CREATIVE_New
    ,a.pi_prod_instnc_stat_cd as status_intial
    , max(pi_cntrct_end_ts) as latest_contract_end_dt

    FROM `divgpras-pr-579355.SHS.SHS_ALR_Campaign_data_May_Aug_2022` b 
    LEFT JOIN`cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht` a
    
    ON a.bacct_bus_bacct_num = b.BACCT_NUM 

    WHERE DATE(prod_instnc_ts) = '2022-05-01'
    and a.pi_prod_instnc_typ_cd='SMHM'
    and a.pi_prod_instnc_stat_cd='A'

    GROUP BY b.CAMP_CREATIVE_New
    , b.BACCT_NUM,a.pi_prod_instnc_stat_cd

)


, Sep_snapshot as (
        
        SELECT
     distinct b.BACCT_NUM as ban
    , b.CAMP_CREATIVE_New as camp_creav_sep
    ,a.pi_prod_instnc_stat_cd as status_Sep
    , max(pi_cntrct_end_ts) as contract_end_dt_sep

    FROM `divgpras-pr-579355.SHS.SHS_ALR_Campaign_data_May_Aug_2022` b 
    LEFT JOIN`cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht` a
    
    ON a.bacct_bus_bacct_num = b.BACCT_NUM 

    WHERE DATE(prod_instnc_ts) = '2022-09-01'
    and a.pi_prod_instnc_typ_cd='SMHM'

    GROUP BY b.CAMP_CREATIVE_New
    , b.BACCT_NUM,a.pi_prod_instnc_stat_cd

)

, Feb_snapshot as (
        
        SELECT
     distinct b.BACCT_NUM as ban
    , b.CAMP_CREATIVE_New as camp_creav_Feb
    ,a.pi_prod_instnc_stat_cd as status_Feb
    , max(pi_cntrct_end_ts) as contract_end_dt_Feb

    FROM `divgpras-pr-579355.SHS.SHS_ALR_Campaign_data_May_Aug_2022` b 
    LEFT JOIN`cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht` a
    
    ON a.bacct_bus_bacct_num = b.BACCT_NUM 

    WHERE DATE(prod_instnc_ts) = '2023-02-01'
    and a.pi_prod_instnc_typ_cd='SMHM'

    GROUP BY b.CAMP_CREATIVE_New
    , b.BACCT_NUM,a.pi_prod_instnc_stat_cd

)




select * from Intial_snapshot a
left JOIN  Sep_snapshot b
on a.ban=b.ban and a.CAMP_CREATIVE_New=b.camp_creav_sep

left join Feb_snapshot c
on a.ban=c.ban and a.CAMP_CREATIVE_New=c.camp_creav_Feb



'''

In [ ]:
Contract_DF=extract_bq_data(bq_client, sql=Contract_status_Query)

In [ ]:
Contract_DF.info()

In [ ]:
Contract_DF.to_csv('Contract_Status.csv',index=False)

In [ ]:
May_2021_prod_mix='''



DECLARE target_date DATE DEFAULT "2022-05-01";
DECLARE interval_days INT64 DEFAULT 0;


WITH cte_max_prod_instnc_date AS(
  SELECT DATE_ADD(target_date,INTERVAL interval_days DAY) AS max_date 
)

, cte_product_mix AS (
  SELECT ffh_prod.bacct_bus_bacct_num AS ban,
         COUNT(DISTINCT CASE WHEN ffh_prod.pi_prod_instnc_typ_cd IN ('SING', 'HSIC', 'TTV', 'SMHM', 'STV', 'DIIC','C') THEN ffh_prod.pi_prod_instnc_typ_cd ELSE NULL END) AS product_mix_all,
         COUNT(DISTINCT CASE WHEN ffh_prod.pi_prod_instnc_typ_cd = 'HSIC' THEN ffh_prod.bus_prod_instnc_id ELSE NULL END) AS hsic_count,
         COUNT(DISTINCT CASE WHEN ffh_prod.pi_prod_instnc_typ_cd = 'SING' THEN ffh_prod.bus_prod_instnc_id ELSE NULL END) AS sing_count,
         COUNT(DISTINCT CASE WHEN ffh_prod.pi_prod_instnc_typ_cd = 'SMHM' THEN ffh_prod.bus_prod_instnc_id ELSE NULL END) AS shs_count,
         COUNT(DISTINCT CASE WHEN ffh_prod.pi_prod_instnc_typ_cd = 'TTV'  THEN ffh_prod.bus_prod_instnc_id ELSE NULL END) AS ttv_count,
         COUNT(DISTINCT CASE WHEN ffh_prod.pi_prod_instnc_typ_cd = 'STV'  THEN ffh_prod.bus_prod_instnc_id ELSE NULL END) AS stv_count,
         COUNT(DISTINCT CASE WHEN ffh_prod.pi_prod_instnc_typ_cd = 'DIIC' THEN ffh_prod.bus_prod_instnc_id ELSE NULL END) AS diic_count,
         COUNT(DISTINCT CASE WHEN ffh_prod.pi_prod_instnc_typ_cd = 'C' THEN ffh_prod.bus_prod_instnc_id ELSE NULL END) AS C_count
   FROM `cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht` AS ffh_prod
   CROSS JOIN cte_max_prod_instnc_date AS dt
   WHERE DATE(ffh_prod.prod_instnc_ts) = dt.max_date
    --  AND ffh_prod.bacct_bus_bacct_num_src_id = 1001 --Wireline
     AND ffh_prod.pi_prod_instnc_stat_cd = 'A' --Active Products
     AND ffh_prod.consldt_cust_typ_cd = 'R' --Regular (not Business)
     AND ffh_prod.pi_prod_instnc_typ_cd IN 
         (
           'DIIC', --Dialup
           'HSIC', --High Speed
           'SING', --Home Phone
           'SMHM', --Smart Home
           'STV',  --Satelite
           'TTV'   --TV
            ,'C'    
         )
   GROUP BY ffh_prod.bacct_bus_bacct_num
)

, Camp_BAN as (

select distinct BACCT_NUM as BAN,CAMP_CREATIVE_New from `divgpras-pr-579355.SHS.SHS_ALR_Campaign_data_May_Aug_2022`

)

select * 
from Camp_BAN a
left join cte_product_mix as b
on a.BAN=b.ban
order by CAMP_CREATIVE_New


'''



In [ ]:
May2022_Prod_Mix_DF=extract_bq_data(bq_client, sql=May_2021_prod_mix)

In [ ]:
May2022_Prod_Mix_DF.info()

In [ ]:
May2022_Prod_Mix_DF.head()

In [ ]:
May2022_Prod_Mix_DF.to_csv('May2022_Prod_Mix.csv',index=False)

In [ ]:
Sep_2022_prod_mix='''



DECLARE target_date DATE DEFAULT "2022-09-01";
DECLARE interval_days INT64 DEFAULT 0;


WITH cte_max_prod_instnc_date AS(
  SELECT DATE_ADD(target_date,INTERVAL interval_days DAY) AS max_date 
)

, cte_product_mix AS (
  SELECT ffh_prod.bacct_bus_bacct_num AS ban,
         COUNT(DISTINCT CASE WHEN ffh_prod.pi_prod_instnc_typ_cd IN ('SING', 'HSIC', 'TTV', 'SMHM', 'STV', 'DIIC','C') THEN ffh_prod.pi_prod_instnc_typ_cd ELSE NULL END) AS product_mix_all,
         COUNT(DISTINCT CASE WHEN ffh_prod.pi_prod_instnc_typ_cd = 'HSIC' THEN ffh_prod.bus_prod_instnc_id ELSE NULL END) AS hsic_count,
         COUNT(DISTINCT CASE WHEN ffh_prod.pi_prod_instnc_typ_cd = 'SING' THEN ffh_prod.bus_prod_instnc_id ELSE NULL END) AS sing_count,
         COUNT(DISTINCT CASE WHEN ffh_prod.pi_prod_instnc_typ_cd = 'SMHM' THEN ffh_prod.bus_prod_instnc_id ELSE NULL END) AS shs_count,
         COUNT(DISTINCT CASE WHEN ffh_prod.pi_prod_instnc_typ_cd = 'TTV'  THEN ffh_prod.bus_prod_instnc_id ELSE NULL END) AS ttv_count,
         COUNT(DISTINCT CASE WHEN ffh_prod.pi_prod_instnc_typ_cd = 'STV'  THEN ffh_prod.bus_prod_instnc_id ELSE NULL END) AS stv_count,
         COUNT(DISTINCT CASE WHEN ffh_prod.pi_prod_instnc_typ_cd = 'DIIC' THEN ffh_prod.bus_prod_instnc_id ELSE NULL END) AS diic_count,
         COUNT(DISTINCT CASE WHEN ffh_prod.pi_prod_instnc_typ_cd = 'C' THEN ffh_prod.bus_prod_instnc_id ELSE NULL END) AS C_count
   FROM `cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht` AS ffh_prod
   CROSS JOIN cte_max_prod_instnc_date AS dt
   WHERE DATE(ffh_prod.prod_instnc_ts) = dt.max_date
     AND ffh_prod.pi_prod_instnc_stat_cd = 'A' --Active Products
     AND ffh_prod.consldt_cust_typ_cd = 'R' --Regular (not Business)
     AND ffh_prod.pi_prod_instnc_typ_cd IN 
         (
           'DIIC', --Dialup
           'HSIC', --High Speed
           'SING', --Home Phone
           'SMHM', --Smart Home
           'STV',  --Satelite
           'TTV'   --TV
            ,'C'    
         )
   GROUP BY ffh_prod.bacct_bus_bacct_num
)

, Camp_BAN as (

select distinct BACCT_NUM as BAN,CAMP_CREATIVE_New from `divgpras-pr-579355.SHS.SHS_ALR_Campaign_data_May_Aug_2022`

)

select * 
from Camp_BAN a
left join cte_product_mix as b
on a.BAN=b.ban
order by CAMP_CREATIVE_New


'''

In [ ]:
Sep2022_Prod_Mix_DF=extract_bq_data(bq_client, sql=Sep_2022_prod_mix)

In [ ]:
Sep2022_Prod_Mix_DF.head()

In [ ]:
Sep2022_Prod_Mix_DF.info()

In [ ]:
Sep2022_Prod_Mix_DF.to_csv('Sep2022_Prod_Mix.csv',index=False)

In [ ]:
Feb_2023_prod_mix='''



DECLARE target_date DATE DEFAULT "2023-02-01";
DECLARE interval_days INT64 DEFAULT 0;


WITH cte_max_prod_instnc_date AS(
  SELECT DATE_ADD(target_date,INTERVAL interval_days DAY) AS max_date 
)

, cte_product_mix AS (
  SELECT ffh_prod.bacct_bus_bacct_num AS ban,
         COUNT(DISTINCT CASE WHEN ffh_prod.pi_prod_instnc_typ_cd IN ('SING', 'HSIC', 'TTV', 'SMHM', 'STV', 'DIIC','C') THEN ffh_prod.pi_prod_instnc_typ_cd ELSE NULL END) AS product_mix_all,
         COUNT(DISTINCT CASE WHEN ffh_prod.pi_prod_instnc_typ_cd = 'HSIC' THEN ffh_prod.bus_prod_instnc_id ELSE NULL END) AS hsic_count,
         COUNT(DISTINCT CASE WHEN ffh_prod.pi_prod_instnc_typ_cd = 'SING' THEN ffh_prod.bus_prod_instnc_id ELSE NULL END) AS sing_count,
         COUNT(DISTINCT CASE WHEN ffh_prod.pi_prod_instnc_typ_cd = 'SMHM' THEN ffh_prod.bus_prod_instnc_id ELSE NULL END) AS shs_count,
         COUNT(DISTINCT CASE WHEN ffh_prod.pi_prod_instnc_typ_cd = 'TTV'  THEN ffh_prod.bus_prod_instnc_id ELSE NULL END) AS ttv_count,
         COUNT(DISTINCT CASE WHEN ffh_prod.pi_prod_instnc_typ_cd = 'STV'  THEN ffh_prod.bus_prod_instnc_id ELSE NULL END) AS stv_count,
         COUNT(DISTINCT CASE WHEN ffh_prod.pi_prod_instnc_typ_cd = 'DIIC' THEN ffh_prod.bus_prod_instnc_id ELSE NULL END) AS diic_count,
         COUNT(DISTINCT CASE WHEN ffh_prod.pi_prod_instnc_typ_cd = 'C' THEN ffh_prod.bus_prod_instnc_id ELSE NULL END) AS C_count
   FROM `cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht` AS ffh_prod
   CROSS JOIN cte_max_prod_instnc_date AS dt
   WHERE DATE(ffh_prod.prod_instnc_ts) = dt.max_date
     AND ffh_prod.pi_prod_instnc_stat_cd = 'A' --Active Products
     AND ffh_prod.consldt_cust_typ_cd = 'R' --Regular (not Business)
     AND ffh_prod.pi_prod_instnc_typ_cd IN 
         (
           'DIIC', --Dialup
           'HSIC', --High Speed
           'SING', --Home Phone
           'SMHM', --Smart Home
           'STV',  --Satelite
           'TTV'   --TV
            ,'C'    
         )
   GROUP BY ffh_prod.bacct_bus_bacct_num
)

, Camp_BAN as (

select distinct BACCT_NUM as BAN,CAMP_CREATIVE_New from `divgpras-pr-579355.SHS.SHS_ALR_Campaign_data_May_Aug_2022`

)

select * 
from Camp_BAN a
left join cte_product_mix as b
on a.BAN=b.ban
order by CAMP_CREATIVE_New


'''

In [ ]:
Feb2023_Prod_Mix_DF=extract_bq_data(bq_client, sql=Feb_2023_prod_mix)

In [ ]:
Feb2023_Prod_Mix_DF.info()

In [ ]:
Feb2023_Prod_Mix_DF.head(10)

In [ ]:
Feb2023_Prod_Mix_DF.to_csv('Feb2023_Prod_Mix.csv',index=False)